# 2D-DMS : Double mutant analysis.


In [1]:
import re
import pandas as pd
import numpy as np
import statistics as st
import itertools as it
import matplotlib.pyplot as plt
import matplotlib as mpl
import py3Dmol as py3D
import bokeh as bk
import pandas_bokeh as pd_bk
from bokeh.models import FixedTicker, ColorBar, LinearColorMapper
pd_bk.output_notebook()


Loading BokehJS ...

In [2]:
partner1 = "VHH.2"
partner2 = "TNF.2"
condition_KanA = "38"

In [3]:
column_name = list()
l_enrichment = list()
l_DOUBLEMUT_ENRICH_series = list()
DOUBLEMUT_ENRICH_file = f"VHH.2-TNF.2_TANDEM_MUT_2_ENRICH.out"

comment_lines = ""
pat = re.compile(r"\$\d+:\s?(.+?)[,\n]")
parse_comments = True
d_DOUBLEMUT_ENRICH_series = dict()
d_ordermut = dict(l_aa1 = [], l_aa2 = [],l_aa1_seen = [], l_aa2_seen = [], l_tandem = [])
s_mutation_not_seen_in_p1 = set()
s_mutation_not_seen_in_p2 = set()
s_mutation_seen_in_p1 = set()
s_mutation_seen_in_p2 = set()
for line in open(DOUBLEMUT_ENRICH_file).readlines():
    if line[0] == "#":
        comment_lines += line[1:]
        continue
    if parse_comments:
        column_name = pat.findall(comment_lines)
        print(column_name)
        parse_comments = False
        
    s = line.split()
    aamut1 = s[0]
    aamut2 = s[1]
    tandem_mut = "{}_{}".format(aamut1, aamut2)
    if aamut1 not in d_ordermut['l_aa1']:
        d_ordermut['l_aa1'].append(aamut1)
    if aamut2 not in d_ordermut['l_aa2']:
        d_ordermut['l_aa2'].append(aamut2)        
    if tandem_mut not in d_ordermut['l_tandem']:
        d_ordermut['l_tandem'].append(tandem_mut)
        
    if len(s) <= 3:
        continue
    s_mutation_seen_in_p1.add(aamut1)
    s_mutation_seen_in_p2.add(aamut2)

    d_DOUBLEMUT_ENRICH_series[tandem_mut] = s

s_mutation_not_seen_in_p1 = set(d_ordermut['l_aa1']) - s_mutation_seen_in_p1
s_mutation_not_seen_in_p2 = set(d_ordermut['l_aa2']) - s_mutation_seen_in_p2
    
    
for aamut1 in d_ordermut['l_aa1']:
    for aamut2 in d_ordermut['l_aa2']:
        tmut = "{}_{}".format(aamut1, aamut2)
        if tmut in d_DOUBLEMUT_ENRICH_series:
            l_DOUBLEMUT_ENRICH_series.append(d_DOUBLEMUT_ENRICH_series[tmut])

df_DOUBLEMUT_ENRICH = pd.DataFrame([pd.Series(l_DOUBLEMUT_ENRICH_series[x],index=column_name) for x in range(len(l_DOUBLEMUT_ENRICH_series))], columns = column_name)
df_DOUBLEMUT_ENRICH.replace('None', np.nan, inplace=True)
#for col in column_name[2:]:

l_property = ['mean enrich tandem mutant', 'mean1 vs WT', 'mean2 vs WT', 'stdev', 'stdev1', 'stdev2', 
              'size in terms of number of bc', 'size1', 'size2']
for prop in l_property:
    df_DOUBLEMUT_ENRICH[prop] = df_DOUBLEMUT_ENRICH[prop].astype(float)

print("Total number of mutations expected in p1: {}".format(len(d_ordermut['l_aa1'])))
print("Number of mutations observed in p1: {}".format(len(s_mutation_seen_in_p1)))
print("Total number of mutations expected in p2: {}".format(len(d_ordermut['l_aa2'])))
print("Number of mutations observed in p2: {}".format(len(s_mutation_seen_in_p2)))

df_DOUBLEMUT_ENRICH

['Aminoacid mutation in partner 1', 'Aminoacid mutation in partner 2', 'mean enrich tandem mutant', 'stdev', 'median', 'quant_1st ', 'quant_2nd', 'mean1 vs WT', 'stdev1', 'median1', 'quant_1st1', 'quant_2nd1', 'mean2 vs WT', 'stdev2', 'median2', 'quant_1st2', 'quant_2nd2', 'size in terms of number of bc', 'size1', 'size2']
Total number of mutations expected in p1: 185
Number of mutations observed in p1: 117
Total number of mutations expected in p2: 190
Number of mutations observed in p2: 102


,Aminoacid mutation in partner 1,Aminoacid mutation in partner 2,mean enrich tandem mutant,stdev,median,quant_1st,quant_2nd,mean1 vs WT,stdev1,median1,quant_1st1,quant_2nd1,mean2 vs WT,stdev2,median2,quant_1st2,quant_2nd2,size in terms of number of bc,size1,size2
0,S30D,R32D,1.15,0.19,1.18,0.96,1.31,1.13,0.20,1.16,1.01,1.26,1.10,0.16,1.09,0.98,1.19,5.0,38.0,61.0
1,S30D,R32E,0.85,0.52,NaN,NaN,NaN,1.13,0.20,1.16,1.01,1.26,0.87,0.40,1.03,0.31,1.19,3.0,38.0,39.0
2,S30D,R32F,0.82,0.07,0.82,0.75,0.89,1.13,0.20,1.16,1.01,1.26,0.77,0.15,0.79,0.69,0.86,4.0,38.0,71.0
3,S30D,R32G,1.12,0.00,NaN,NaN,NaN,1.13,0.20,1.16,1.01,1.26,0.98,0.12,0.96,0.88,1.05,1.0,38.0,19.0
4,S30D,R32K,1.19,0.00,NaN,NaN,NaN,1.13,0.20,1.16,1.01,1.26,0.99,0.20,0.93,0.84,1.13,1.0,38.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9044,F103V,S81F,0.10,0.00,NaN,NaN,NaN,0.22,0.03,NaN,NaN,NaN,0.18,0.06,0.17,0.13,0.2,1.0,2.0,58.0
9045,F103V,Q88K,0.13,0.00,NaN,NaN,NaN,0.22,0.03,NaN,NaN,NaN,0.95,0.18,0.95,0.85,1.03,1.0,2.0,27.0
9046,F103V,K90N,0.00,0.00,NaN,NaN,NaN,0.22,0.03,NaN,NaN,NaN,0.11,0.03,0.11,0.09,0.12,1.0,2.0,26.0
9047,F103V,N92S,0.77,0.00,NaN,NaN,NaN,0.22,0.03,NaN,NaN,NaN,1.09,0.15,1.04,1.01,1.17,1.0,2.0,22.0


In [4]:
def get_color(parameter, ):
    
    if float(parameter) == -0.1:
        color = '#eeeeee'
    elif float(parameter) >= 0 and float(parameter) <= 0.2:
        color = '#3b19ff'  # Deep blue
    elif float(parameter) > 0.2 and float(parameter) <= 0.4:
        color = '#6fa8dc' # Pale Blue
    elif float(parameter) > 0.4 and float(parameter) <= 0.6:
        color = '#93c47d'  # Pale Green
    elif float(parameter) > 0.6 and float(parameter) <= 0.8:
        color = '#ffe599'   # Pale Yellow
    elif float(parameter) > 0.8 and float(parameter) <= 1.:
        color = '#f1c232'   # Yellow
    elif float(parameter) > 1. and float(parameter) <= 1.2:
        color = '#e69138'  # Orange
    elif float(parameter) > 1.2 and float(parameter) <= 1.4:
        color = '#d63636'  # Deep Red
    elif float(parameter) > 1.4:
        color = '#69031a'  # Dark mauve
    else:
        color = None
    return color


## Global analysis of the tandem mutations ##

In [5]:
"""
Data preparation
"""

# Graphic analysis for the enrichment of mutations.
l_position_vector_1 = list()
l_mutation_vector_1 = list()
l_position_vector_2 = list()
l_mutation_vector_2  = list()
s_aa_1 = set()
s_aa_2 = set()

#for row in df_DOUBLEMUT_ENRICH.itertuples():
for mutaa1 in d_ordermut['l_aa1']:
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    d_ordermut['l_aa1_seen'].append(mutaa1)
    
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    #print(row)
    if mutaa1[1:-1] not in l_position_vector_1:
        #print(row[1][1:-1])
        l_position_vector_1.append(mutaa1[1:-1])
        s_aa_1.add(mutaa1[:-1])                 
    if mutaa1 not in l_mutation_vector_1:
        l_mutation_vector_1.append(mutaa1)
for mutaa2 in d_ordermut['l_aa2']:
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    d_ordermut['l_aa2_seen'].append(mutaa2)
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    if mutaa2[1:-1] not in l_position_vector_2:
        l_position_vector_2.append(mutaa2[1:-1])
        s_aa_2.add(mutaa2[:-1]) 
    if mutaa2 not in l_mutation_vector_2:
        l_mutation_vector_2.append(mutaa2)

print("Number of mutations in partner 1: {}".format(len(l_mutation_vector_1)))

print("Number of mutations in partner 2: {}".format(len(l_mutation_vector_2)))


enrich_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))
enrich_mutation_array += -0.1

nbc_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))


mut_xaxis_partner2 = list()
mut2vsWT_xaxis_partner2 = list()
mut_yaxis_partner1 = list()
mu1vsWT_yaxis_partner1 = list()

aa_mut_yaxis_partner1 = list()
aa_mut_xaxis_partner2 = list()

color = list()
print(len(color))
alpha = list()
df = df_DOUBLEMUT_ENRICH

color.append("black")
alpha.append(1.)
aa_mut_yaxis_partner1.append("WT")
aa_mut_xaxis_partner2.append("WT")
mut_yaxis_partner1.append("WT")
mut_xaxis_partner2.append("WT")
mu1vsWT_yaxis_partner1.append(1)
mut2vsWT_xaxis_partner2.append(1)
d_mutvsWT = dict(p1=dict(), p2=dict())

for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
    df_block = df.loc[(df['Aminoacid mutation in partner 2'] == elt2)]
    try:
        enr_mut2vsWT = df_block.iloc[0]['mean2 vs WT']
        stdev_mut2vsWT = df_block.iloc[0]['stdev2']
        size2_mut2vsWT = df_block.iloc[0]['size2']
    except IndexError:
        enr_mut2vsWT = -0.1
        stdev_mut2vsWT = -0.1
        size2_mut2vsWT = 0.        
    enrich_mutation_array[0, jj+1] = "{:.2f}".format(enr_mut2vsWT)
    nbc_mutation_array[0, jj+1] = min(10., size2_mut2vsWT)
    alpha.append(1.)
    color.append(get_color(enr_mut2vsWT))
    mut_xaxis_partner2.append(elt2)
    mut_yaxis_partner1.append("WT")
    mu1vsWT_yaxis_partner1.append("{:.2f}".format(enr_mut2vsWT))
    mut2vsWT_xaxis_partner2.append("-")
    d_mutvsWT['p2'][elt2] = dict()
    d_mutvsWT['p2'][elt2]['enrich'] = enr_mut2vsWT
    d_mutvsWT['p2'][elt2]['stdev'] = stdev_mut2vsWT
    
for ii, elt in enumerate(l_mutation_vector_1):#l_mutation_vector_1):

    # All lines of df_block contain the same info for mean1. We take [0] as a sample
    df_block = df.loc[(df['Aminoacid mutation in partner 1'] == elt)]
    try:
        enr_mut1vsWT = df_block.iloc[0]['mean1 vs WT']
        stdev_mut1vsWT = df_block.iloc[0]['stdev1']
        size1_mut1vsWT = df_block.iloc[0]['size1']
    except IndexError:
        enr_mut1vsWT = -0.1
        stdev_mut1vsWT = -0.1
        size1_mut1vsWT = 0.
    #print(ii, elt, enr_mut1vsWT, get_color(enr_mut1vsWT))
    alpha.append(1.)
    color.append(get_color(enr_mut1vsWT))
    enrich_mutation_array[ii+1, 0] = "{:.2f}".format(enr_mut1vsWT)
    nbc_mutation_array[ii+1, 0] = min(10., size1_mut1vsWT)
    
    mut_xaxis_partner2.append("WT")
    mut_yaxis_partner1.append(elt)
    mu1vsWT_yaxis_partner1.append("-")
    mut2vsWT_xaxis_partner2.append("{:.2f}".format(enr_mut1vsWT))
    d_mutvsWT['p1'][elt] = dict()
    d_mutvsWT['p1'][elt]['enrich'] = enr_mut1vsWT
    d_mutvsWT['p1'][elt]['stdev'] = stdev_mut1vsWT
    
    for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
        
        df_row = df_block.loc[(df_block['Aminoacid mutation in partner 2'] == elt2)]
        
        counter_id_xaxis = 0
        #if str(df_row['Aminoacid mutation in partner 1']) == elt and str(df_row['Aminoacid mutation in partner 2']) == elt2:
        #mut_xaxis_partner2.append(row[4])
        #aa_mut_xaxis_partner2.append(row[2])
        #mut_yaxis_partner1.append(elt)
        #aa_mut_yaxis_partner1.append(row[1])
        mut_xaxis_partner2.append(elt2)
        mut_yaxis_partner1.append(elt)
        try:
            mu1vsWT_yaxis_partner1.append("{:.2f}".format(d_mutvsWT['p2'][elt2]['enrich']))
            mut2vsWT_xaxis_partner2.append("{:.2f}".format(d_mutvsWT['p1'][elt]['enrich']))
        except KeyError:
            mu1vsWT_yaxis_partner1.append("-0.1")
            mut2vsWT_xaxis_partner2.append("-0.1")
        try:
            enrichment = df_row.iloc[0]['mean enrich tandem mutant']
            nbc = df_row.iloc[0]['size in terms of number of bc']
        except IndexError:
            enrichment = -0.1
            nbc = 0.
        
        alpha.append(1.)
        enrich_mutation_array[ii+1, jj+1] = "{:.2f}".format(enrichment)
        nbc_mutation_array[ii+1, jj+1] = min(10., nbc)
        color.append(get_color(enrichment))

#print("color", len(color))
#print(color)


Number of mutations in partner 1: 117
Number of mutations in partner 2: 102
0


In [6]:
"""
Data plotting
"""
#path = "./Enrichment.png"

data_tot=dict(
    xax = mut_xaxis_partner2,
    yax = mut_yaxis_partner1,
    colors = color,
    enrich = enrich_mutation_array.flatten(),
    alphas = alpha,
    mut2vsWT_xax = mu1vsWT_yaxis_partner1, 
    mut1vsWT_yax = mut2vsWT_xaxis_partner2,
    nbc_size = nbc_mutation_array.flatten()/10.,
)

    #aa_mut_xax = aa_mut_xaxis_partner2,
    #aa_mut_yax = aa_mut_yaxis_partner1,
p = bk.plotting.figure(title="Effects of double mutations according to the enrichment factors", 
                       x_axis_location="above", tools="hover, save, box_zoom, pan, reset",
                       x_range=["WT"]+l_mutation_vector_2, y_range=list(reversed(["WT"]+l_mutation_vector_1)),
                       tooltips = [
                           ('Mut partner 1', '@yax'), 
                           ('Mut partner 2', '@xax'), 
                           ('Enrichments', '@enrich'), 
                           ('Mut in Partner1 vs WT', '@mut1vsWT_yax'),
                           ('Mut in Partner2 vs WT', '@mut2vsWT_xax'), 
                           
                       ]
                      )
p.width = 900
p.height = 900
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3
p.yaxis.axis_label = partner1
p.xaxis.axis_label = partner2
#p.image_url(url=[path])

"""
To modulate the size of the boxes wth respect to the number of barcodes : 
--> switch the comments between the two following lines
"""

#p.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color='colors', line_color=None, alpha='alphas', 
#       hover_line_color='black', hover_color='colors')
p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color='colors', line_color=None, alpha='alphas', 
       hover_line_color='black', hover_color='colors')

mapper = LinearColorMapper(palette=['#3b19ff','#6fa8dc','#93c47d','#ffe599','#f1c232','#e69138','#d63636','#69031a'], low=0, high=1.6)
tick=[0,0.2,0.4,0.6,0.8,1.0,1.2,1.4]
color_bar = ColorBar(title="Enrichment", height=300, width = 10, color_mapper=mapper, ticker=FixedTicker(ticks=tick, desired_num_ticks = len(tick)),
                     label_standoff=12, border_line_color=None, location=(0,0), 
                     orientation="vertical")

p.add_layout(color_bar, 'right')

bk.plotting.show(p)

<img src="./Type1_rescue.png" alt="rescue1" width="700"/>


In [7]:
"""
Data preparation
"""

# Graphic analysis for the enrichment of mutations.
l_position_vector_1 = list()
l_mutation_vector_1 = list()
l_position_vector_2 = list()
l_mutation_vector_2  = list()
s_aa_1 = set()
s_aa_2 = set()

#for row in df_DOUBLEMUT_ENRICH.itertuples():
for mutaa1 in d_ordermut['l_aa1']:
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    d_ordermut['l_aa1_seen'].append(mutaa1)
    
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    #print(row)
    if mutaa1[1:-1] not in l_position_vector_1:
        #print(row[1][1:-1])
        l_position_vector_1.append(mutaa1[1:-1])
        s_aa_1.add(mutaa1[:-1])                 
    if mutaa1 not in l_mutation_vector_1:
        l_mutation_vector_1.append(mutaa1)
for mutaa2 in d_ordermut['l_aa2']:
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    d_ordermut['l_aa2_seen'].append(mutaa2)
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    if mutaa2[1:-1] not in l_position_vector_2:
        l_position_vector_2.append(mutaa2[1:-1])
        s_aa_2.add(mutaa2[:-1]) 
    if mutaa2 not in l_mutation_vector_2:
        l_mutation_vector_2.append(mutaa2)

print("Number of mutations in partner 1: {}".format(len(l_mutation_vector_1)))

print("Number of mutations in partner 2: {}".format(len(l_mutation_vector_2)))


enrich_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))
enrich_mutation_array += -0.1

nbc_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))


mut_xaxis_partner2 = list()
mut2vsWT_xaxis_partner2 = list()
mut_yaxis_partner1 = list()
mu1vsWT_yaxis_partner1 = list()
color = list()
print(len(color))
alpha = list()
df = df_DOUBLEMUT_ENRICH

color.append("black")
alpha.append(1.)
aa_mut_yaxis_partner1.append("WT")
aa_mut_xaxis_partner2.append("WT")
mut_yaxis_partner1.append("WT")
mut_xaxis_partner2.append("WT")
mu1vsWT_yaxis_partner1.append(1)
mut2vsWT_xaxis_partner2.append(1)
d_mutvsWT = dict(p1=dict(), p2=dict())

for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
    df_block = df.loc[(df['Aminoacid mutation in partner 2'] == elt2)]
    try:
        enr_mut2vsWT = df_block.iloc[0]['mean2 vs WT']
        stdev_mut2vsWT = df_block.iloc[0]['stdev2']
        size2_mut2vsWT = df_block.iloc[0]['size2']
    except IndexError:
        enr_mut2vsWT = -0.1
        stdev_mut2vsWT = -0.1
        size2_mut2vsWT = 0.        
    enrich_mutation_array[0, jj+1] = "{:.2f}".format(enr_mut2vsWT)
    nbc_mutation_array[0, jj+1] = min(10., size2_mut2vsWT)
    alpha.append(1.)
    color.append(get_color(enr_mut2vsWT))
    mut_xaxis_partner2.append(elt2)
    mut_yaxis_partner1.append("WT")
    mu1vsWT_yaxis_partner1.append("{:.2f}".format(enr_mut2vsWT))
    mut2vsWT_xaxis_partner2.append("-")
    d_mutvsWT['p2'][elt2] = dict()
    d_mutvsWT['p2'][elt2]['enrich'] = enr_mut2vsWT
    d_mutvsWT['p2'][elt2]['stdev'] = stdev_mut2vsWT
    
for ii, elt in enumerate(l_mutation_vector_1):#l_mutation_vector_1):

    # All lines of df_block contain the same info for mean1. We take [0] as a sample
    df_block = df.loc[(df['Aminoacid mutation in partner 1'] == elt)]
    try:
        enr_mut1vsWT = df_block.iloc[0]['mean1 vs WT']
        stdev_mut1vsWT = df_block.iloc[0]['stdev1']
        size1_mut1vsWT = df_block.iloc[0]['size1']
    except IndexError:
        enr_mut1vsWT = -0.1
        stdev_mut1vsWT = -0.1
        size1_mut1vsWT = 0.
    #print(ii, elt, enr_mut1vsWT, get_color(enr_mut1vsWT))
    alpha.append(1.)
    color.append(get_color(enr_mut1vsWT))
    enrich_mutation_array[ii+1, 0] = "{:.2f}".format(enr_mut1vsWT)
    nbc_mutation_array[ii+1, 0] = min(10., size1_mut1vsWT)
    
    mut_xaxis_partner2.append("WT")
    mut_yaxis_partner1.append(elt)
    mu1vsWT_yaxis_partner1.append("-")
    mut2vsWT_xaxis_partner2.append("{:.2f}".format(enr_mut1vsWT))
    d_mutvsWT['p1'][elt] = dict()
    d_mutvsWT['p1'][elt]['enrich'] = enr_mut1vsWT
    d_mutvsWT['p1'][elt]['stdev'] = stdev_mut1vsWT
    
    for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
        
        df_row = df_block.loc[(df_block['Aminoacid mutation in partner 2'] == elt2)]
        
        counter_id_xaxis = 0
        mut_xaxis_partner2.append(elt2)
        mut_yaxis_partner1.append(elt)
        try:
            mu1vsWT_yaxis_partner1.append("{:.2f}".format(d_mutvsWT['p2'][elt2]['enrich']))
            mut2vsWT_xaxis_partner2.append("{:.2f}".format(d_mutvsWT['p1'][elt]['enrich']))
        except KeyError:
            mu1vsWT_yaxis_partner1.append("-0.1")
            mut2vsWT_xaxis_partner2.append("-0.1")
        try:
            enrichment = df_row.iloc[0]['mean enrich tandem mutant']
            nbc = df_row.iloc[0]['size in terms of number of bc']
        except IndexError:
            enrichment = -0.1
            nbc = 0.
        
        alpha.append(1.)
        
        p1 = d_mutvsWT['p1'][elt]['enrich']
        p2 = d_mutvsWT['p2'][elt2]['enrich']
        
        """
        Conditions to define the scores are defined here  
        """
        
        if p1 <1 and p2 <1:
            diff = (enrichment - min(p1,p2))*(1-min(p1,p2))
            if diff < 0:
                diff = 0
        elif p1 >1 and p2 >1:
            diff =  min(p1,p2) - enrichment
            #if diff > 0:
            diff  = 0
        else:
            diff = 0
            
        
        enrich_mutation_array[ii+1, jj+1] = "{:.2f}".format(diff)
        nbc_mutation_array[ii+1, jj+1] = min(10., nbc) # to modulate the size of the box
        color.append(get_color(diff))

#print("color", len(color))
#print(color)

Number of mutations in partner 1: 117
Number of mutations in partner 2: 102
0


In [8]:
"""
Data plotting
"""
# Interesting cases W33R-N92D


from bokeh.transform import linear_cmap
from bokeh.palettes import Reds6

data_tot=dict(
    xax = mut_xaxis_partner2,
    yax = mut_yaxis_partner1,
    colors = color,
    enrich = enrich_mutation_array.flatten(),
    alphas = alpha,
    mut2vsWT_xax = mu1vsWT_yaxis_partner1, 
    mut1vsWT_yax = mut2vsWT_xaxis_partner2,
    nbc_size = nbc_mutation_array.flatten()/10.,
)

    #aa_mut_xax = aa_mut_xaxis_partner2,
    #aa_mut_yax = aa_mut_yaxis_partner1,
p = bk.plotting.figure(title="Effects of double mutations according to the enrichment factors", 
                       x_axis_location="above", tools="hover, save, box_zoom, pan, reset",
                       x_range=["WT"]+l_mutation_vector_2, y_range=list(reversed(["WT"]+l_mutation_vector_1)),
                       tooltips = [
                           ('Mut partner 1', '@yax'), 
                           ('Mut partner 2', '@xax'), 
                           ('Diff', '@enrich'), 
                           ('Mut in Partner1 vs WT', '@mut1vsWT_yax'),
                           ('Mut in Partner2 vs WT', '@mut2vsWT_xax'), 
                           
                       ]
                      )
p.width = 900
p.height = 900
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3
p.yaxis.axis_label = partner1
p.xaxis.axis_label = partner2

"""
To modulate the size of the boxes wth respect to the number of barcodes : 
--> switch the comments between the two following lines
"""

p.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color=linear_cmap('enrich', palette=Reds6, low=0.5, high=0), line_color=None, alpha='alphas', 
       hover_line_color='black', hover_color='colors')
#p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color=linear_cmap('enrich', palette=Reds6, low=0.5, high=0), line_color=None, alpha='alphas', 
#       hover_line_color='black', hover_color='colors')

bk.plotting.show(p)

## Specific analysis between pairs of residue (VHH2-W33R vs TNF2-N92D)

In [9]:
## Display all the possible residues ##
print("Mutated AA in partner 1:\n",s_aa_1)
print("Mutated AA in partner 2:\n",s_aa_2)

Mutated AA in partner 1:
 {'S30', 'T53', 'I57', 'Y35', 'E50', 'N74', 'Y32', 'W33', 'S101', 'N31', 'F103', 'N54', 'R98', 'L56', 'S99'}
Mutated AA in partner 2:
 {'L75', 'T89', 'E135', 'N92', 'S81', 'E146', 'I97', 'R32', 'N137', 'T79', 'T77', 'K90', 'S95', 'S147', 'Q88'}


In [10]:
residue1 = "33"
residue2 = "92"
df_select = df_DOUBLEMUT_ENRICH[df_DOUBLEMUT_ENRICH["Aminoacid mutation in partner 1"].str.contains(residue1)]
df_select2 = df_select[df_select["Aminoacid mutation in partner 2"].str.contains(residue2)]

df3 = df_select2.copy()

df3["mean enrich tandem mutant"] = df3["mean enrich tandem mutant"].astype(float)



In [11]:
"""
Data preparation
"""

# Graphic analysis for the enrichment of mutations.
l_position_vector_1 = list()
l_mutation_vector_1 = list()
l_position_vector_2 = list()
l_mutation_vector_2  = list()
s_aa_1 = set()
s_aa_2 = set()

#for row in df_DOUBLEMUT_ENRICH.itertuples():
for mutaa1 in d_ordermut['l_aa1']:
    pos_mutaa1 = mutaa1[1:-1]
    if pos_mutaa1 != residue1:
        continue
    d_ordermut['l_aa1_seen'].append(mutaa1)
    
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    #print(row)
    if mutaa1[1:-1] not in l_position_vector_1:
        #print(row[1][1:-1])
        l_position_vector_1.append(mutaa1[1:-1])
        s_aa_1.add(mutaa1[:-1])                 
    if mutaa1 not in l_mutation_vector_1:
        l_mutation_vector_1.append(mutaa1)
for mutaa2 in d_ordermut['l_aa2']:
    pos_mutaa2 = mutaa2[1:-1]
    if pos_mutaa2 != residue2:
        continue
    d_ordermut['l_aa2_seen'].append(mutaa2)
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    if mutaa2[1:-1] not in l_position_vector_2:
        l_position_vector_2.append(mutaa2[1:-1])
        s_aa_2.add(mutaa2[:-1]) 
    if mutaa2 not in l_mutation_vector_2:
        l_mutation_vector_2.append(mutaa2)

print("Number of mutations in partner 1: {}".format(len(l_mutation_vector_1)))

print("Number of mutations in partner 2: {}".format(len(l_mutation_vector_2)))


enrich_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))
enrich_mutation_array += -0.1

nbc_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))

mut_xaxis_partner2 = list()
mut2vsWT_xaxis_partner2 = list()
mut_yaxis_partner1 = list()
mu1vsWT_yaxis_partner1 = list()
color = list()
print(len(color))
alpha = list()
df = df_DOUBLEMUT_ENRICH

color.append("black")
alpha.append(1.)
aa_mut_yaxis_partner1.append("WT")
aa_mut_xaxis_partner2.append("WT")
mut_yaxis_partner1.append("WT")
mut_xaxis_partner2.append("WT")
mu1vsWT_yaxis_partner1.append(1)
mut2vsWT_xaxis_partner2.append(1)
d_mutvsWT = dict(p1=dict(), p2=dict())

for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
    df_block = df.loc[(df['Aminoacid mutation in partner 2'] == elt2)]
    try:
        enr_mut2vsWT = df_block.iloc[0]['mean2 vs WT']
        stdev_mut2vsWT = df_block.iloc[0]['stdev2']
        size2_mut2vsWT = df_block.iloc[0]['size2']
    except IndexError:
        enr_mut2vsWT = -0.1
        stdev_mut2vsWT = -0.1
        size2_mut2vsWT = 0.        
    enrich_mutation_array[0, jj+1] = "{:.2f}".format(enr_mut2vsWT)
    nbc_mutation_array[0, jj+1] = min(10., size2_mut2vsWT)
    alpha.append(1.)
    color.append(get_color(enr_mut2vsWT))
    mut_xaxis_partner2.append(elt2)
    mut_yaxis_partner1.append("WT")
    mu1vsWT_yaxis_partner1.append("{:.2f}".format(enr_mut2vsWT))
    mut2vsWT_xaxis_partner2.append("-")
    d_mutvsWT['p2'][elt2] = dict()
    d_mutvsWT['p2'][elt2]['enrich'] = enr_mut2vsWT
    d_mutvsWT['p2'][elt2]['stdev'] = stdev_mut2vsWT
    
for ii, elt in enumerate(l_mutation_vector_1):#l_mutation_vector_1):

    # All lines of df_block contain the same info for mean1. We take [0] as a sample
    df_block = df.loc[(df['Aminoacid mutation in partner 1'] == elt)]
    try:
        enr_mut1vsWT = df_block.iloc[0]['mean1 vs WT']
        stdev_mut1vsWT = df_block.iloc[0]['stdev1']
        size1_mut1vsWT = df_block.iloc[0]['size1']
    except IndexError:
        enr_mut1vsWT = -0.1
        stdev_mut1vsWT = -0.1
        size1_mut1vsWT = 0.
    #print(ii, elt, enr_mut1vsWT, get_color(enr_mut1vsWT))
    alpha.append(1.)
    color.append(get_color(enr_mut1vsWT))
    enrich_mutation_array[ii+1, 0] = "{:.2f}".format(enr_mut1vsWT)
    nbc_mutation_array[ii+1, 0] = min(10., size1_mut1vsWT)
    
    mut_xaxis_partner2.append("WT")
    mut_yaxis_partner1.append(elt)
    mu1vsWT_yaxis_partner1.append("-")
    mut2vsWT_xaxis_partner2.append("{:.2f}".format(enr_mut1vsWT))
    d_mutvsWT['p1'][elt] = dict()
    d_mutvsWT['p1'][elt]['enrich'] = enr_mut1vsWT
    d_mutvsWT['p1'][elt]['stdev'] = stdev_mut1vsWT
    
    for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
        
        df_row = df_block.loc[(df_block['Aminoacid mutation in partner 2'] == elt2)]
        
        counter_id_xaxis = 0
        #if str(df_row['Aminoacid mutation in partner 1']) == elt and str(df_row['Aminoacid mutation in partner 2']) == elt2:
        #mut_xaxis_partner2.append(row[4])
        #aa_mut_xaxis_partner2.append(row[2])
        #mut_yaxis_partner1.append(elt)
        #aa_mut_yaxis_partner1.append(row[1])
        mut_xaxis_partner2.append(elt2)
        mut_yaxis_partner1.append(elt)
        try:
            mu1vsWT_yaxis_partner1.append("{:.2f} +/- {:.2f}".format(d_mutvsWT['p2'][elt2]['enrich'], 
                                                                      d_mutvsWT['p2'][elt2]['stdev']))
            mut2vsWT_xaxis_partner2.append("{:.2f} +/- {:.2f}".format(d_mutvsWT['p1'][elt]['enrich'],
                                                                     d_mutvsWT['p1'][elt]['stdev']))
        except KeyError:
            mu1vsWT_yaxis_partner1.append("-0.1")
            mut2vsWT_xaxis_partner2.append("-0.1")
        try:
            enrichment = df_row.iloc[0]['mean enrich tandem mutant']
            stdev = df_row.iloc[0]['stdev']
            nbc = df_row.iloc[0]['size in terms of number of bc']
        except IndexError:
            enrichment = -0.1
            stdev = 0
            nbc = 0.
        
        alpha.append(1.)
        enrich_mutation_array[ii+1, jj+1] = "{:.2f}".format(enrichment)
        nbc_mutation_array[ii+1, jj+1] = min(10., nbc)
        color.append(get_color(enrichment))

print("color", len(color))


Number of mutations in partner 1: 10
Number of mutations in partner 2: 11
0
color 132


In [12]:
"""
Data plotting
"""
data_tot=dict(
    xax=mut_xaxis_partner2,
    yax=mut_yaxis_partner1,
    colors=color,
    enrich=enrich_mutation_array.flatten(),
    alphas = alpha,
    mut2vsWT_xax = mu1vsWT_yaxis_partner1, 
    mut1vsWT_yax = mut2vsWT_xaxis_partner2,
    nbc_size = nbc_mutation_array.flatten()/10.,
    nbc_int = nbc_mutation_array.flatten(),
)

    #aa_mut_xax = aa_mut_xaxis_partner2,
    #aa_mut_yax = aa_mut_yaxis_partner1,
p = bk.plotting.figure(title="Effects of double mutations according to the enrichment factors", 
                       x_axis_location="above", tools="hover, save, box_zoom, pan, reset",
                       x_range=["WT"]+l_mutation_vector_2, y_range=list(reversed(["WT"]+l_mutation_vector_1)),
                       tooltips = [
                           ('Mut partner 1', '@yax'), 
                           ('Mut partner 2', '@xax'), 
                           ('Enrichments', '@enrich'), 
                           ('Mut in Partner1 vs WT', '@mut1vsWT_yax'),
                           ('Mut in Partner2 vs WT', '@mut2vsWT_xax'), 
                           ('Nbc', '@nbc_int'),
                           
                       ]
                      )
p.width = min(1000, len(l_mutation_vector_2)*80)
p.height = min(1000, len(l_mutation_vector_1)*80)
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "14px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3
p.yaxis.axis_label = partner1
p.xaxis.axis_label = partner2

"""
To modulate the size of the boxes wth respect to the number of barcodes : 
--> switch the comments between the two following lines
"""
#p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color='colors', line_color='lightgray', alpha='alphas', 
#       hover_line_color='black', hover_color='colors')

p.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color='colors', line_color='lightgray', alpha='alphas', 
       hover_line_color='black', hover_color='colors')

bk.plotting.show(p)

# Vizualisation (VHH2-W33R en interaction avec TNF-N92D)

In [13]:
plt = py3D.view(query='pdb:5m2j')
chA = {'chain':'A'}
plt.zoomTo()
plt.setStyle(chA,{'cartoon': {'color':'purple'}})
plt.addStyle({'chain':'A','resi':92},{'stick':{'color':'blue'}})
plt.addSurface(py3D.VDW,{'opacity':0.6,'color':'white'}, chA)
plt.addLabel("TNF-N92D(blue)",{'fontOpacity':10},{'resi':155})
chD = {'chain':'D'}
plt.setStyle(chD,{'cartoon': {'color':'yellow'}})
plt.addStyle({'chain':'D','resi':33},{'stick':{'color':'red'}})
#plt.addStyle({'within':{'distance': 3, 'sel':{'resi':33}}},{'stick':{'colorscheme':'grayCarbon'}})
plt.addSurface(py3D.VDW,{'opacity':0.7,'color':'white'}, chD)
plt.addLabel("VHH2-W33R(red)",{'fontOpacity':10},{'resi':1})

plt.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Formation d'un nouveau pont salin

<img src="./W33R_N92D.png" alt="W33R_N92D" width="900"/>

<img src="./Type2_amplification.png" alt="rescue2" width="400"/>

## Analysis taking into account WT values ##
### Analysis of highly stabilized pairs for individually favorable mutations ###

In [14]:
"""
Data preparation
"""

# Graphic analysis for the enrichment of mutations.
l_position_vector_1 = list()
l_mutation_vector_1 = list()
l_position_vector_2 = list()
l_mutation_vector_2  = list()
s_aa_1 = set()
s_aa_2 = set()

#for row in df_DOUBLEMUT_ENRICH.itertuples():
for mutaa1 in d_ordermut['l_aa1']:
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    d_ordermut['l_aa1_seen'].append(mutaa1)
    
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    #print(row)
    if mutaa1[1:-1] not in l_position_vector_1:
        #print(row[1][1:-1])
        l_position_vector_1.append(mutaa1[1:-1])
        s_aa_1.add(mutaa1[:-1])                 
    if mutaa1 not in l_mutation_vector_1:
        l_mutation_vector_1.append(mutaa1)
for mutaa2 in d_ordermut['l_aa2']:
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    d_ordermut['l_aa2_seen'].append(mutaa2)
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    if mutaa2[1:-1] not in l_position_vector_2:
        l_position_vector_2.append(mutaa2[1:-1])
        s_aa_2.add(mutaa2[:-1]) 
    if mutaa2 not in l_mutation_vector_2:
        l_mutation_vector_2.append(mutaa2)

print("Number of mutations in partner 1: {}".format(len(l_mutation_vector_1)))

print("Number of mutations in partner 2: {}".format(len(l_mutation_vector_2)))


enrich_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))
enrich_mutation_array += -0.1

nbc_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))


mut_xaxis_partner2 = list()
mut2vsWT_xaxis_partner2 = list()
mut_yaxis_partner1 = list()
mu1vsWT_yaxis_partner1 = list()
color = list()
print(len(color))
alpha = list()
df = df_DOUBLEMUT_ENRICH

color.append("black")
alpha.append(1.)
aa_mut_yaxis_partner1.append("WT")
aa_mut_xaxis_partner2.append("WT")
mut_yaxis_partner1.append("WT")
mut_xaxis_partner2.append("WT")
mu1vsWT_yaxis_partner1.append(1)
mut2vsWT_xaxis_partner2.append(1)
d_mutvsWT = dict(p1=dict(), p2=dict())

for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
    df_block = df.loc[(df['Aminoacid mutation in partner 2'] == elt2)]
    try:
        enr_mut2vsWT = df_block.iloc[0]['mean2 vs WT']
        stdev_mut2vsWT = df_block.iloc[0]['stdev2']
        size2_mut2vsWT = df_block.iloc[0]['size2']
    except IndexError:
        enr_mut2vsWT = -0.1
        stdev_mut2vsWT = -0.1
        size2_mut2vsWT = 0.        
    enrich_mutation_array[0, jj+1] = "{:.2f}".format(enr_mut2vsWT)
    nbc_mutation_array[0, jj+1] = min(10., size2_mut2vsWT)
    alpha.append(1.)
    color.append(get_color(enr_mut2vsWT))
    mut_xaxis_partner2.append(elt2)
    mut_yaxis_partner1.append("WT")
    mu1vsWT_yaxis_partner1.append("{:.2f}".format(enr_mut2vsWT))
    mut2vsWT_xaxis_partner2.append("-")
    d_mutvsWT['p2'][elt2] = dict()
    d_mutvsWT['p2'][elt2]['enrich'] = enr_mut2vsWT
    d_mutvsWT['p2'][elt2]['stdev'] = stdev_mut2vsWT
    
for ii, elt in enumerate(l_mutation_vector_1):#l_mutation_vector_1):

    # All lines of df_block contain the same info for mean1. We take [0] as a sample
    df_block = df.loc[(df['Aminoacid mutation in partner 1'] == elt)]
    try:
        enr_mut1vsWT = df_block.iloc[0]['mean1 vs WT']
        stdev_mut1vsWT = df_block.iloc[0]['stdev1']
        size1_mut1vsWT = df_block.iloc[0]['size1']
    except IndexError:
        enr_mut1vsWT = -0.1
        stdev_mut1vsWT = -0.1
        size1_mut1vsWT = 0.
    #print(ii, elt, enr_mut1vsWT, get_color(enr_mut1vsWT))
    alpha.append(1.)
    color.append(get_color(enr_mut1vsWT))
    enrich_mutation_array[ii+1, 0] = "{:.2f}".format(enr_mut1vsWT)
    nbc_mutation_array[ii+1, 0] = min(10., size1_mut1vsWT)
    
    mut_xaxis_partner2.append("WT")
    mut_yaxis_partner1.append(elt)
    mu1vsWT_yaxis_partner1.append("-")
    mut2vsWT_xaxis_partner2.append("{:.2f}".format(enr_mut1vsWT))
    d_mutvsWT['p1'][elt] = dict()
    d_mutvsWT['p1'][elt]['enrich'] = enr_mut1vsWT
    d_mutvsWT['p1'][elt]['stdev'] = stdev_mut1vsWT
    
    for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
        
        df_row = df_block.loc[(df_block['Aminoacid mutation in partner 2'] == elt2)]
        
        counter_id_xaxis = 0
        mut_xaxis_partner2.append(elt2)
        mut_yaxis_partner1.append(elt)
        try:
            mu1vsWT_yaxis_partner1.append("{:.2f}".format(d_mutvsWT['p2'][elt2]['enrich']))
            mut2vsWT_xaxis_partner2.append("{:.2f}".format(d_mutvsWT['p1'][elt]['enrich']))
        except KeyError:
            mu1vsWT_yaxis_partner1.append("-0.1")
            mut2vsWT_xaxis_partner2.append("-0.1")
        try:
            enrichment = df_row.iloc[0]['mean enrich tandem mutant']
            nbc = df_row.iloc[0]['size in terms of number of bc']
        except IndexError:
            enrichment = -0.1
            nbc = 0.
        
        alpha.append(1.)
        
        p1 = d_mutvsWT['p1'][elt]['enrich']
        p2 = d_mutvsWT['p2'][elt2]['enrich']

        if p1 <1 or p2 <1:
            diff = 0
        elif p1 >1 and p2 >1:
            diff =  enrichment - max(p1, p2)
            if diff < 0:
                diff  = 0
        else:
            diff = 0
        enrich_mutation_array[ii+1, jj+1] = "{:.2f}".format(diff)
        nbc_mutation_array[ii+1, jj+1] = min(10., nbc)
        color.append(get_color(diff))

#print("color", len(color))
#print(color)

Number of mutations in partner 1: 117
Number of mutations in partner 2: 102
0


In [15]:
"""
Data plotting
"""

# Interesting mutations : S30K - S147D
#                         S101Q - T89F

from bokeh.transform import linear_cmap
from bokeh.palettes import Greens8

data_tot=dict(
    xax = mut_xaxis_partner2,
    yax = mut_yaxis_partner1,
    colors = color,
    enrich = enrich_mutation_array.flatten(),
    alphas = alpha,
    mut2vsWT_xax = mu1vsWT_yaxis_partner1, 
    mut1vsWT_yax = mut2vsWT_xaxis_partner2,
    nbc_size = nbc_mutation_array.flatten()/10.,
)

    #aa_mut_xax = aa_mut_xaxis_partner2,
    #aa_mut_yax = aa_mut_yaxis_partner1,
p = bk.plotting.figure(title="Effects of double mutations according to the enrichment factors", 
                       x_axis_location="above", tools="hover, save, box_zoom, pan, reset",
                       x_range=["WT"]+l_mutation_vector_2, y_range=list(reversed(["WT"]+l_mutation_vector_1)),
                       tooltips = [
                           ('Mut partner 1', '@yax'), 
                           ('Mut partner 2', '@xax'), 
                           ('Diff', '@enrich'), 
                           ('Mut in Partner1 vs WT', '@mut1vsWT_yax'),
                           ('Mut in Partner2 vs WT', '@mut2vsWT_xax'), 
                           
                       ]
                      )
p.width = 800
p.height = 800
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3
p.yaxis.axis_label = partner1
p.xaxis.axis_label = partner2

"""
To modulate the size of the boxes wth respect to the number of barcodes : 
--> switch the comments between the two following lines
"""

#p.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color=linear_cmap('enrich', palette=Reds6, low=1, high=0), line_color=None, alpha='alphas', 
#       hover_line_color='black', hover_color='colors')
p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color=linear_cmap('enrich', palette=Greens8, low=1., high=0), line_color=None, alpha='alphas', 
       hover_line_color='black', hover_color='colors')

bk.plotting.show(p)

# Vizualisation (VHH2-S30K en interaction avec TNF-S147D)

In [16]:
"""
residue1 = "30"
residue2 = "147"
df_select = df_DOUBLEMUT_ENRICH[df_DOUBLEMUT_ENRICH["Aminoacid mutation in partner 1"].str.contains(residue1)]
df_select2 = df_select[df_select["Aminoacid mutation in partner 2"].str.contains(residue2)]

df3 = df_select2.copy()

df3["mean enrich tandem mutant"] = df3["mean enrich tandem mutant"].astype(float)
"""

"""
Data preparation
"""

"""
# Graphic analysis for the enrichment of mutations.
l_position_vector_1 = list()
l_mutation_vector_1 = list()
l_position_vector_2 = list()
l_mutation_vector_2  = list()
s_aa_1 = set()
s_aa_2 = set()

#for row in df_DOUBLEMUT_ENRICH.itertuples():
for mutaa1 in d_ordermut['l_aa1']:
    pos_mutaa1 = mutaa1[1:-1]
    if pos_mutaa1 != residue1:
        continue
    d_ordermut['l_aa1_seen'].append(mutaa1)
    
    if mutaa1 not in s_mutation_seen_in_p1:
        continue
    #print(row)
    if mutaa1[1:-1] not in l_position_vector_1:
        #print(row[1][1:-1])
        l_position_vector_1.append(mutaa1[1:-1])
        s_aa_1.add(mutaa1[:-1])                 
    if mutaa1 not in l_mutation_vector_1:
        l_mutation_vector_1.append(mutaa1)
for mutaa2 in d_ordermut['l_aa2']:
    pos_mutaa2 = mutaa2[1:-1]
    if pos_mutaa2 != residue2:
        continue
    d_ordermut['l_aa2_seen'].append(mutaa2)
    if mutaa2 not in s_mutation_seen_in_p2:
        continue
    if mutaa2[1:-1] not in l_position_vector_2:
        l_position_vector_2.append(mutaa2[1:-1])
        s_aa_2.add(mutaa2[:-1]) 
    if mutaa2 not in l_mutation_vector_2:
        l_mutation_vector_2.append(mutaa2)

print("Number of mutations in partner 1: {}".format(len(l_mutation_vector_1)))

print("Number of mutations in partner 2: {}".format(len(l_mutation_vector_2)))


enrich_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))
enrich_mutation_array += -0.1

nbc_mutation_array = np.zeros((len(l_mutation_vector_1)+1, len(l_mutation_vector_2)+1))

mut_xaxis_partner2 = list()
mut2vsWT_xaxis_partner2 = list()
mut_yaxis_partner1 = list()
mu1vsWT_yaxis_partner1 = list()
color = list()
print(len(color))
alpha = list()
df = df_DOUBLEMUT_ENRICH

color.append("black")
alpha.append(1.)
aa_mut_yaxis_partner1.append("WT")
aa_mut_xaxis_partner2.append("WT")
mut_yaxis_partner1.append("WT")
mut_xaxis_partner2.append("WT")
mu1vsWT_yaxis_partner1.append(1)
mut2vsWT_xaxis_partner2.append(1)
d_mutvsWT = dict(p1=dict(), p2=dict())

for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
    df_block = df.loc[(df['Aminoacid mutation in partner 2'] == elt2)]
    try:
        enr_mut2vsWT = df_block.iloc[0]['mean2 vs WT']
        stdev_mut2vsWT = df_block.iloc[0]['stdev2']
        size2_mut2vsWT = df_block.iloc[0]['size2']
    except IndexError:
        enr_mut2vsWT = -0.1
        stdev_mut2vsWT = -0.1
        size2_mut2vsWT = 0.        
    enrich_mutation_array[0, jj+1] = "{:.2f}".format(enr_mut2vsWT)
    nbc_mutation_array[0, jj+1] = min(10., size2_mut2vsWT)
    alpha.append(1.)
    color.append(get_color(enr_mut2vsWT))
    mut_xaxis_partner2.append(elt2)
    mut_yaxis_partner1.append("WT")
    mu1vsWT_yaxis_partner1.append("{:.2f}".format(enr_mut2vsWT))
    mut2vsWT_xaxis_partner2.append("-")
    d_mutvsWT['p2'][elt2] = dict()
    d_mutvsWT['p2'][elt2]['enrich'] = enr_mut2vsWT
    d_mutvsWT['p2'][elt2]['stdev'] = stdev_mut2vsWT
    
for ii, elt in enumerate(l_mutation_vector_1):#l_mutation_vector_1):

    # All lines of df_block contain the same info for mean1. We take [0] as a sample
    df_block = df.loc[(df['Aminoacid mutation in partner 1'] == elt)]
    try:
        enr_mut1vsWT = df_block.iloc[0]['mean1 vs WT']
        stdev_mut1vsWT = df_block.iloc[0]['stdev1']
        size1_mut1vsWT = df_block.iloc[0]['size1']
    except IndexError:
        enr_mut1vsWT = -0.1
        stdev_mut1vsWT = -0.1
        size1_mut1vsWT = 0.
    #print(ii, elt, enr_mut1vsWT, get_color(enr_mut1vsWT))
    alpha.append(1.)
    color.append(get_color(enr_mut1vsWT))
    enrich_mutation_array[ii+1, 0] = "{:.2f}".format(enr_mut1vsWT)
    nbc_mutation_array[ii+1, 0] = min(10., size1_mut1vsWT)
    
    mut_xaxis_partner2.append("WT")
    mut_yaxis_partner1.append(elt)
    mu1vsWT_yaxis_partner1.append("-")
    mut2vsWT_xaxis_partner2.append("{:.2f}".format(enr_mut1vsWT))
    d_mutvsWT['p1'][elt] = dict()
    d_mutvsWT['p1'][elt]['enrich'] = enr_mut1vsWT
    d_mutvsWT['p1'][elt]['stdev'] = stdev_mut1vsWT
    
    for jj, elt2 in enumerate(l_mutation_vector_2):#l_mutation_vector_2):
        
        df_row = df_block.loc[(df_block['Aminoacid mutation in partner 2'] == elt2)]
        
        counter_id_xaxis = 0
        #if str(df_row['Aminoacid mutation in partner 1']) == elt and str(df_row['Aminoacid mutation in partner 2']) == elt2:
        #mut_xaxis_partner2.append(row[4])
        #aa_mut_xaxis_partner2.append(row[2])
        #mut_yaxis_partner1.append(elt)
        #aa_mut_yaxis_partner1.append(row[1])
        mut_xaxis_partner2.append(elt2)
        mut_yaxis_partner1.append(elt)
        try:
            mu1vsWT_yaxis_partner1.append("{:.2f} +/- {:.2f}".format(d_mutvsWT['p2'][elt2]['enrich'], 
                                                                      d_mutvsWT['p2'][elt2]['stdev']))
            mut2vsWT_xaxis_partner2.append("{:.2f} +/- {:.2f}".format(d_mutvsWT['p1'][elt]['enrich'],
                                                                     d_mutvsWT['p1'][elt]['stdev']))
        except KeyError:
            mu1vsWT_yaxis_partner1.append("-0.1")
            mut2vsWT_xaxis_partner2.append("-0.1")
        try:
            enrichment = df_row.iloc[0]['mean enrich tandem mutant']
            stdev = df_row.iloc[0]['stdev']
            nbc = df_row.iloc[0]['size in terms of number of bc']
        except IndexError:
            enrichment = -0.1
            stdev = 0
            nbc = 0.
        
        alpha.append(1.)
        enrich_mutation_array[ii+1, jj+1] = "{:.2f}".format(enrichment)
        nbc_mutation_array[ii+1, jj+1] = min(10., nbc)
        color.append(get_color(enrichment))

print("color", len(color))
"""

"""
Data plotting
"""

"""
data_tot=dict(
    xax=mut_xaxis_partner2,
    yax=mut_yaxis_partner1,
    colors=color,
    enrich=enrich_mutation_array.flatten(),
    alphas = alpha,
    mut2vsWT_xax = mu1vsWT_yaxis_partner1, 
    mut1vsWT_yax = mut2vsWT_xaxis_partner2,
    nbc_size = nbc_mutation_array.flatten()/10.,
    nbc_int = nbc_mutation_array.flatten(),
)

    #aa_mut_xax = aa_mut_xaxis_partner2,
    #aa_mut_yax = aa_mut_yaxis_partner1,
p = bk.plotting.figure(title="Effects of double mutations according to the enrichment factors", 
                       x_axis_location="above", tools="hover, save, box_zoom, pan, reset",
                       x_range=["WT"]+l_mutation_vector_2, y_range=list(reversed(["WT"]+l_mutation_vector_1)),
                       tooltips = [
                           ('Mut partner 1', '@yax'), 
                           ('Mut partner 2', '@xax'), 
                           ('Enrichments', '@enrich'), 
                           ('Mut in Partner1 vs WT', '@mut1vsWT_yax'),
                           ('Mut in Partner2 vs WT', '@mut2vsWT_xax'), 
                           ('Nbc', '@nbc_int'),
                           
                       ]
                      )
p.width = min(1000, len(l_mutation_vector_2)*80)
p.height = min(1000, len(l_mutation_vector_1)*80)
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "14px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3
p.yaxis.axis_label = partner1
p.xaxis.axis_label = partner2

"""

"""
To modulate the size of the boxes wth respect to the number of barcodes : 
--> switch the comments between the two following lines
"""

"""
#p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color='colors', line_color='lightgray', alpha='alphas', 
#       hover_line_color='black', hover_color='colors')

p.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color='colors', line_color='lightgray', alpha='alphas', 
       hover_line_color='black', hover_color='colors')

bk.plotting.show(p)
"""



"\n#p.rect('xax', 'yax', 0.9, 0.9, source=data_tot, color='colors', line_color='lightgray', alpha='alphas', \n#       hover_line_color='black', hover_color='colors')\n\np.rect('xax', 'yax', 'nbc_size', 'nbc_size', source=data_tot, color='colors', line_color='lightgray', alpha='alphas', \n       hover_line_color='black', hover_color='colors')\n\nbk.plotting.show(p)\n"

In [17]:
plt = py3D.view(query='pdb:5m2j')
chA = {'chain':'A'}
plt.zoomTo()
plt.setStyle(chA,{'cartoon': {'color':'purple'}})
plt.addStyle({'chain':'A','resi':147},{'stick':{'color':'blue'}})
plt.addSurface(py3D.VDW,{'opacity':0.6,'color':'white'}, chA)
plt.addLabel("TNF-S147D(blue)",{'fontOpacity':10},{'resi':155})
chD = {'chain':'D'}
plt.setStyle(chD,{'cartoon': {'color':'yellow'}})
plt.addStyle({'chain':'D','resi':30},{'stick':{'color':'red'}})
#plt.addStyle({'within':{'distance': 3, 'sel':{'resi':33}}},{'stick':{'colorscheme':'grayCarbon'}})
plt.addSurface(py3D.VDW,{'opacity':0.7,'color':'white'}, chD)
plt.addLabel("VHH2-S30K(red)",{'fontOpacity':10},{'resi':1})

plt.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# En fait il faut considérer une autre chaine TNF du trimère
### Cela démontre la formation du trimère

In [18]:
plt = py3D.view()
plt.addModel(open('complex.pdb', 'r').read(),'pdb')
chA = {'chain':'A'}
plt.zoomTo()
plt.setStyle(chA,{'cartoon': {'color':'purple'}})
plt.addStyle({'chain':'A','resi':147},{'stick':{'color':'blue'}})
plt.addSurface(py3D.VDW,{'opacity':0.6,'color':'white'}, chA)
plt.addLabel("TNF-S147D(blue)",{'fontOpacity':10},{'resi':155})
chD = {'chain':'D'}
plt.setStyle(chD,{'cartoon': {'color':'yellow'}})
plt.addStyle({'chain':'D','resi':30},{'stick':{'color':'red'}})
#plt.addStyle({'within':{'distance': 3, 'sel':{'resi':33}}},{'stick':{'colorscheme':'grayCarbon'}})
plt.addSurface(py3D.VDW,{'opacity':0.7,'color':'white'}, chD)
plt.addLabel("VHH2-S30K(red)",{'fontOpacity':10},{'resi':1})

plt.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Formation d'un pont salin

<img src="./S30K_S147D.png" alt="S30K_S147D" width="700"/>

# ------------------------------------------------------------------------------  
## Aide à la résolution de modèles de docking

<img src="./Alphafold2.png" alt="docking" width="200"/>
<img src="./Haddock_WODMS_1.png" alt="docking" width="600"/>
<img src="./Haddock_1D_DMS_1.png" alt="docking" width="600"/>

# ------------------------------------------------------------------------------
## With only 2D-DMS
<img src="./Haddock_2D.png" alt="docking" width="800"/>